This notebook evaluates Grad-CAM with UCF-50 frame using the tf-keras-vis implementation. 

This notebook is based on the example presented in tf-keras-vis (Kubota, Y. (2021). tf-keras-vis (Version 0.8.1) [Computer software]. https://keisen.github.io/tf-keras-vis-docs/)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import cv2

Load frames and masks

In [ ]:
!cp -R /content/gdrive/MyDrive/masked_images_backup/masks /
!cp -R /content/gdrive/MyDrive/video_frames/ /

In [ ]:
!pip install tf-keras-vis

     |████████████████████████████████| 53 kB 1.5 MB/s 


In [ ]:
import tf_keras_vis

In [ ]:
model = keras.models.load_model('/content/gdrive/MyDrive/resnet50_v2_transferLearned_model')

In [ ]:
from keras.preprocessing.image import load_img
from keras.applications.resnet_v2 import preprocess_input

In [ ]:
from PIL import Image

In [ ]:
classes_list = ['PizzaTossing', 'PlayingGuitar', 'PushUps', 'SalsaSpin']

In [ ]:
from matplotlib import cm
from tf_keras_vis.gradcam import Gradcam

In [ ]:
def prepare_data(class_name, dir_name, check_preds=False):

  images =[]
  file_names =[]
  masks = []

  masks_root = '/masks'
  images_root ='/video_frames'
  
  
  for mask_name in os.listdir(os.path.join(masks_root, class_name, dir_name, 'masks')):
    image_name = mask_name.replace('png','jpg')
    image_path = os.path.join(images_root,class_name,dir_name,image_name)
    mask_path = os.path.join(masks_root, class_name, dir_name, 'masks', mask_name)

    im = load_img(image_path,target_size=(100,100))
    im = np.array(im)
    x = np.asarray([im])
    x = preprocess_input(x)
    pred = model.predict(x)

    if not os.path.exists('/gradCAM'):
      os.mkdir('/gradCAM')
    if not os.path.exists(os.path.join('/gradCAM',class_name)):
      os.mkdir(os.path.join('/gradCAM',class_name))
    if not os.path.exists(os.path.join('/gradCAM', class_name, dir_name)):
      os.mkdir(os.path.join('/gradCAM', class_name, dir_name))
    

    if classes_list[np.argmax(pred[0])] == class_name or check_preds:

      file_names.append(image_path)



      msk = np.array(Image.open(mask_path).resize(im.shape[1::-1], Image.BILINEAR))
      msk = msk/255

      images.append(im)
      masks.append(msk)
    

  images = np.asarray(images)

  # Preparing input data
  X = preprocess_input(images)
  

  return images, masks, file_names, X

In [ ]:
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
from keras import backend as K
import time

In [ ]:
def create_gradCAM(class_name, dir_name, check_preds = False):
  images, masks, file_names, X = prepare_data(class_name, dir_name, check_preds=check_preds)
  replace2linear = ReplaceToLinear()
  # 0 = PizzaTossing, 1 = PlayingGuiar, 2 = PushUps, 3 = SalsaSpin.
  for index, name in enumerate(classes_list):
    if name==class_name:
      score = CategoricalScore([index])
  start = time.time()
  # Create Gradcam object
  gradcam = Gradcam(model,
                  model_modifier=replace2linear,
                  clone=True)
  

  # Generate heatmap with GradCAM
  cam = gradcam(score,
              X,
              penultimate_layer=-1)
  duration = time.time()-start

  for index, map in enumerate(cam):
    path = file_names[index].replace('video_frames','gradCAM')

    # Render
    f, ax = plt.subplots(nrows=1, ncols=1, figsize=(4, 4))
    #heatmap = np.uint8(cm.jet(map)[..., :3] * 255)
    ax.set_title(class_name, fontsize=16)
    ax.imshow(images[index])
    ax.imshow(map, cmap='jet', alpha=0.5)
    ax.axis('off')
    plt.tight_layout()
    plt.savefig(path)
    #plt.show()
    plt.close()

  return cam, images, masks, file_names, duration


In [ ]:
def evaluate_maps(maps, masks, duration):
  duration = duration/ len(maps)
  duration_list = []
  duration_list.extend([duration]*len(maps))

  map_weight_list = []
  mask_weight_list = []
  for index, map in enumerate(maps):
    map_weight = np.sum(map)
    dst = map * masks[index]
    mask_weight = np.sum(dst)

    map_weight_list.append(map_weight)
    mask_weight_list.append(mask_weight)

  return map_weight_list, mask_weight_list, duration_list



In [ ]:
pizza = [('PizzaTossing','v_PizzaTossing_g02_c02')]
salsa = [('SalsaSpin','v_SalsaSpin_g04_c02'),('SalsaSpin','v_SalsaSpin_g11_c02'),('SalsaSpin','v_SalsaSpin_g18_c04')]
guitar = [('PlayingGuitar', 'v_PlayingGuitar_g06_c05'), ('PlayingGuitar', 'v_PlayingGuitar_g11_c02'),('PlayingGuitar','v_PlayingGuitar_g22_c03')]
pushups = [('PushUps', 'v_PushUps_g11_c03'), ('PushUps','v_PushUps_g14_c04'),('PushUps','v_PushUps_g18_c02')]

#all_classes = [pizza, salsa, guitar, pushups]

In [ ]:
map_weights = []
mask_weights = []
durations = []
all_classes = [pizza, salsa]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_gradCAM(name, dir)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/ResNet50'):
  os.mkdir('/content/gdrive/MyDrive/ResNet50')
!cp -R /gradCAM /content/gdrive/MyDrive/ResNet50/

map_weights = []
mask_weights = []
durations = []
all_classes = [pizza]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_gradCAM(name, dir)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')

map_weights = []
mask_weights = []
durations = []
all_classes = [salsa]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_gradCAM(name, dir)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')


map_weights = []
mask_weights = []
durations = []
all_classes = [pushups]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_gradCAM(name, dir, check_preds = True)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/ResNet50/PushUps'):
  os.mkdir('/content/gdrive/MyDrive/ResNet50/PushUps')
!cp -R /gradCAM /content/gdrive/MyDrive/ResNet50/PushUps/

map_weights = []
mask_weights = []
durations = []
all_classes = [guitar]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_gradCAM(name, dir, check_preds=True)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/ResNet50/PlayingGuitar'):
  os.mkdir('/content/gdrive/MyDrive/ResNet50/PlayingGuitar')
!cp -R /gradCAM /content/gdrive/MyDrive/ResNet50/PlayingGuitar/


map_weights = []
mask_weights = []
durations = []
all_classes = [pizza]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_gradCAM(name, dir, check_preds=True)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/ResNet50/PizzaTossing'):
  os.mkdir('/content/gdrive/MyDrive/ResNet50/PizzaTossing')
!cp -R /gradCAM /content/gdrive/MyDrive/ResNet50/PizzaTossing/


map_weights = []
mask_weights = []
durations = []
all_classes = [salsa]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_gradCAM(name, dir, check_preds=True)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/ResNet50/SalsaSpin'):
  os.mkdir('/content/gdrive/MyDrive/ResNet50/SalsaSpin')
!cp -R /gradCAM /content/gdrive/MyDrive/ResNet50/SalsaSpin/


map_weights = []
mask_weights = []
durations = []
all_classes = [pizza, salsa, guitar, pushups]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_gradCAM(name, dir, check_preds=True)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')


Average mask weight:  912.8329762267201
Standard deviation:  248.1390856640082
Average total weight:  4932.054528130426
Standard deviation:  700.08466
Average computing time:  0.35364146696196636
Standard deviation:  0.11587033662536045
Mask weight percentage:  18.50816877672973 %

Average mask weight:  847.8910201444318
Standard deviation:  60.566935693610645
Average total weight:  5106.980052806713
Standard deviation:  402.7877
Average computing time:  0.21502962377336307
Standard deviation:  2.7755575615628914e-17
Mask weight percentage:  16.60259118651628 %

Average mask weight:  1107.6588444735844
Standard deviation:  429.7411666587364
Average total weight:  4407.2779541015625
Standard deviation:  1051.938
Average computing time:  0.4918509324391683
Standard deviation:  0.1458333801080753
Mask weight percentage:  25.132493480307033 %

Average mask weight:  1067.983438933211
Standard deviation:  708.2267860588169
Average total weight:  3660.112311469184
Standard deviation:  1575.01